We are going to analyse the number of atoms in a MOT using fluoresence imaging.

This is a VERY rough estimate and should only be trusted to within an order of magnitude or so

We proceed following Heathers `Simplified system for creating a BEC`

In [7]:
# Define SI unit prefixes
nm = nHz = nA = nV = nW = 1e-9
um = uHz = uA = uV = uW = 1e-6
mm = mHz = mA = mV = mW = 1e-3
Hz = A = V = W = 1
kHz = kA = kV = kW = 1e3
MHz = MA = MV = MW = 1e6
GHz = GA = GV = GW = 1e9

In [8]:
# automatic generation of SI unit prefixes
# m, Hz, A, V, W
# n, u, m, [], k, M, G

for unit in ['m', 'Hz', 'A', 'V', 'W']:
    for prefix in ['n', 'u', 'm', '', 'k', 'M', 'G']:      
        exec(str(prefix + unit + " = 1e" + str(3 * (["n", "u", "m", "", "k", "M", "G"].index(prefix) - 3))))

In [9]:
import numpy as np

MHz = 1e6
nm = 1e-9
mm = 1e-3
cm = 1e-2
mW = 1e-3
W = 1
V = 1
A = 1
inch = 2.54*cm

# fundamental constants
I_SAT = 3.846*mW/cm**2

# MOT constants
D2_LINEWIDTH = 6.065*MHz
TRAP_WAVELENGTH = 780.24*nm
UNCOATED_INTERFACES = 0

# photodiode constants
RESPONSIVITY = 0.585  # A/W
GAIN = 4.75e6 # V/A in high-Z

In [10]:
def scatteringRate(Io,Isat,detuning,linewidth = D2_LINEWIDTH):
    """
    Calculate `R` the scattering rate of atoms in the MOT.

    All values should be in SI units.
    """
    I = Io/Isat
    return I*np.pi*linewidth / (1 + I + 4*(detuning/linewidth)**2)

In [11]:
def solidAngle(lensDiameter, distanceFromMOT):
    """
    Calculate the solid angle of a lens of diameter `lensDiameter` at a distance `distanceFromMOT` from the MOT.

    All values should be in SI units.
    """
    lensArea = np.pi * (lensDiameter/2)**2
    totalArea = 4 * np.pi * distanceFromMOT**2
    return 4 * np.pi * lensArea / totalArea


minimal = solidAngle(1 * inch, 15 * cm)
maximal = solidAngle(1 * inch, 16 * cm)
print("min 1 inch -> 15cm: ",minimal)
print("max 1 inch -> 16cm: ",maximal)
print(f"variation: {abs(maximal-minimal)/minimal*100:.1f}%")

min 1 inch -> 15cm:  0.022520332404333234
max 1 inch -> 16cm:  0.019793260902246007
variation: 12.1%


In [12]:
def photonEnergy(laserWavelength):
    """
    Calculate the energy of a photon with wavelength `laserWavelength`.

    All values should be in SI units.
    """
    return 6.62607015e-34 * 2.99792458e8 / laserWavelength


In [13]:
def photoCurrent(voltage, gain = GAIN):
    """
    Calculate the photocurrent from a photodiode with voltage `voltage`.

    All values should be in SI units.
    """
    return voltage / gain

In [14]:
def I0(totalPower):
    """
    Calculate the intensity of the trapping laser

    we assume the totalPower is shared across 4 beams, then two are retroreflected

    Each beam is assumed to be a 1 inch circle

    All values should be in SI units.
    """
    totalArea = 6 * np.pi * (0.5 * inch)**2
    return 6/4 * totalPower / totalArea

In [ ]:
VOLTAGE_DIFF = (3.57 - 3.34) * V
TRAPPING_POWER = 75 * mW

solidangle = solidAngle(1 * inch, 15 * cm)
photocurrent = photoCurrent(VOLTAGE_DIFF)
photonenergy = photonEnergy(TRAP_WAVELENGTH)
scatteringrate = scatteringRate(I0(TRAPPING_POWER), I_SAT, 10 * MHz)

print(f"solid angle: {solidangle:.3f} sr")
print(f"photocurrent: {photocurrent:.2e} A")
print(f"photon energy: {photonenergy:.2e} J")
print(f"scattering rate: {scatteringrate/MHz:.2f} MHz")

solid angle: 0.023 sr
photocurrent: 2.38e-07 A
photon energy: 2.55e-19 J
scattering rate: 1.43 MHz


In [16]:
atomNumber = 4 * np.pi * photocurrent
atomNumber /= solidangle * RESPONSIVITY * photonenergy * scatteringrate * 0.96**UNCOATED_INTERFACES

print(f"{atomNumber:.2e} atoms in the MOT")

6.24e+08 atoms in the MOT
